### 웹 크롤링(Web crwaling) 
- 웹페이지를 발견하고 인덱싱
- url , 하이퍼링크 페이지 확인

### 웹 스크래핑
- 웹 페이지로부터 데이터 추출
- 특정 사이트 콘텐츠를 구조화된 형식으로 변환
- HTML 문서에서 필요한 데이터 추출

### 크롤링 + 스크래핑
- URL 생성 -> 다운로드 -> 데이터 추출 -> 데이터 저장

URL을 생성 하는 과정에서 웹의 최상위에 존재하는 robots.txt 라는  파일을 찾고 그안에 프로토콜에 따라 적혀있는 옵션 확인


### Paser 종류
- json -> json
- xml -> xmltodict
- html -> beautifulSoup

## robots.txt 분석하기

In [2]:
# User-agent가 특정 URL에 접근 가능한지 체크
import urllib.robotparser
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://www.reuters.com/robots.txt')
rp.read()

rp.can_fetch(useragent='*', url='https://www.reuters.com/sitemap.xml')  #특정 useragent 가 url에 접근 가능한지 확인

True

### 연습문제(sitemap url을 확보)

In [3]:
# XML 파서를 위한 모듈
import xmltodict
import requests

sitemaps = rp.site_maps()
sitemaps 
# 사이트 들어가보면 URL 정보를 갖는 loc , 최근 수정 시각정보를 갖는 lastmod 있음


url = sitemaps[0]
sitemaps = xmltodict.parse(requests.get(url).text)   # xmltodict을 이용한 xml형식일떄 sitemaps 추출

a= sitemaps['sitemapindex']['sitemap']
k = [ a[i]['loc'] for i in range(len(a)) ]     # sitemaps라는 큰 틀에서 여러 페이지들 추출 
k[:5]

['https://www.reuters.com/arc/outboundfeeds/sitemap/?outputType=xml',
 'https://www.reuters.com/arc/outboundfeeds/sitemap/?outputType=xml&from=100',
 'https://www.reuters.com/arc/outboundfeeds/sitemap/?outputType=xml&from=200',
 'https://www.reuters.com/arc/outboundfeeds/sitemap/?outputType=xml&from=300',
 'https://www.reuters.com/arc/outboundfeeds/sitemap/?outputType=xml&from=400']

In [9]:
url = k[0]     # 여러 페이지중에서 하나를 가지고 와서
news_dict = xmltodict.parse(requests.get(url).text)   
b = news_dict['urlset']['url']              # 그 페이지의 url 값들을 추출
k2 = [ b[i]['loc'] for i in range(len(b)) ] 
k2[:5]

['https://www.reuters.com/markets/europe/bayer-sees-lower-2023-operating-profit-cost-inflation-2023-02-28/',
 'https://www.reuters.com/business/finance/oaktree-capital-seeks-raise-10-billion-new-fund-ft-2023-02-28/',
 'https://www.reuters.com/markets/europe/global-markets-view-europe-2023-02-28/',
 'https://www.reuters.com/business/retail-consumer/home-zara-fast-slow-fashion-collide-2023-02-28/',
 'https://www.reuters.com/lifestyle/sports/england-captain-stokes-has-no-regrets-over-follow-on-2023-02-28/']

## - html 파일 만들기

In [14]:
session = requests.Session()

for url in k2[:5]:
    file = url.split('/')[-2] + '.html'    # /로 분리해서 확장자 html 붙임 
    
    response = session.get(url)
    if response.ok:               # 상태 200 체크하는거랑 비슷
        with open('datas/'+file , 'w+b') as f:
            f.write(response.text.encode('utf-8'))
    else:
        print('error with URL :'.format(url))


In [15]:
file = open('filename.txt','w') # 파일을 생성하고 w(쓰기모드)
file.write('파일에 작성할 문자열') # 파일에 문자열 작성
file.close() # 처리 끝나면 닫아주기 

In [16]:
with open('filename.txt','r') as file:  # 파일 읽어오기 r(읽기보드)
    print(file.read())

파일에 작성할 문자열


In [17]:
import os
path = './datas/'
files = [path + file for file in os.listdir(path)] # os.listdir(path) -> 폴더 안에있는 파일들 목록 가져옴
files

['./datas/bayer-sees-lower-2023-operating-profit-cost-inflation-2023-02-28.html',
 './datas/england-captain-stokes-has-no-regrets-over-follow-on-2023-02-28.html',
 './datas/global-markets-view-europe-2023-02-28.html',
 './datas/home-zara-fast-slow-fashion-collide-2023-02-28.html',
 './datas/oaktree-capital-seeks-raise-10-billion-new-fund-ft-2023-02-28.html']

- BeautifulSoup

In [ ]:
with open(files[2], 'r',encoding='utf-8') as f :
    html = f.read()
html

In [39]:
## html은 BeautifulSoup

from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')
title = soup.select_one('#main-content > article > div.article__main__33WV2 > div.article__content__6hMn9 > header > div > div > h1')
# select_one 이 아니라 select면 해당되는 것들 리스트로 가지고옴(one을 쓰면 해당되는것중 가장 첫번째를 불러온다)
print(title.text) # 제목 불러오기 

Morning Bid: Market mood downbeat ahead of raft of data


In [33]:
soup.h1 # .h1 을 조회하면 <h1>태그에 해당하는 객체를 가져올 수 있음

<h1 class="text__text__1FZLe text__dark-grey__3Ml43 text__medium__1kbOh text__heading_3__1kDhc heading__base__2T28j heading__heading_3__3aL54 article-header__title__3Y2hh" data-testid="Heading">Morning Bid: Market mood downbeat ahead of raft of data</h1>

In [34]:
soup.h1.text # 태그 사이에 있는 텍스트 값을 가져올 수 있음 

'Morning Bid: Market mood downbeat ahead of raft of data'

In [44]:
soup.select_one('#main-content > article > div.article__main__33WV2 > div.article__content__6hMn9 > div > div > div.article-body__content__17Yit.paywall-article > p:nth-child(2)').text
# 본문 중 일부 불러오기 

'A look at the day ahead in European and global markets from Anshuman Daga'

In [41]:
r = soup.select('#main-content > article > div.article__main__33WV2 > div.article__content__6hMn9 > div > div > div.article-body__content__17Yit.paywall-article > p')

r = [x.text for x in r]
r

['A look at the day ahead in European and global markets from Anshuman Daga',
 "The overarching downbeat mood among investors shows no signs of improving as markets become increasingly wary of a further rise in borrowing costs.Although U.S. markets took a breather and rose on Monday, they ended well below the day's highs and Asian markets were back in the red on Tuesday after gaining in early trade.",
 "Tuesday's U.S. consumer confidence data will be especially scrutinised for households' views on economic prospects and inflation expectations.",
 'Economists polled by Reuters expect a median reading of 109.5 on the index, which unexpectedly fell in January.',
 'European markets will deal with CPI data due from France and Spain.',
 'Despite sharp increases in interest rates by major central banks, predictions of economies worldwide falling into recession continue to miss the mark.',
 'While inflation has eased a bit, providing some support to markets, a barrage of economic data suggests

In [45]:
soup.find('link',{'rel':'canonical'})['href']
# 표준(canoncical URL)을 사용하여 값을 관리

'https://www.reuters.com/markets/europe/global-markets-view-europe-2023-02-28/'

### HTML 문서의 표준화된 영역

- 제목 : title 또는 h1 태그를 사용
- 웹 페이지 요약 : meta name='description'
- 구조화된 헤더 정보: OpenGraph(https://ogp.me)에서 표준화한 것으로 페이지의 소스코드에서 og: 부분을 찾는다. 
- 웹 페이지의 URL: link rel=”canonical”